In [26]:
import torch
from torch.optim import Adam

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

import cv2
import numpy as np
import json

Using device: cuda


In [27]:
with open("./data/calibration_data.json", "r") as f:
    calib_data = json.load(f)

with open("./data/coordinates_data.json", "r") as f:
    coord_data = json.load(f)

mtx = torch.tensor(calib_data["mtx"], dtype=torch.float32)
dist = torch.tensor(calib_data["dist"], dtype=torch.float32)
objpoints = torch.tensor(coord_data["objpoints"], dtype=torch.float32)
imgpoints = torch.tensor(coord_data["imgpoints"], dtype=torch.float32)
rvecs = torch.tensor(calib_data["rvecs"], dtype=torch.float32)
tvecs = torch.tensor(calib_data["tvecs"], dtype=torch.float32)

In [28]:
def reprojection_error(rvecs, tvecs, objpoints, imgpoints, mtx, dist):
    total_error = 0
    for i in range(len(objpoints)):
        imgpoints2, _ = cv2.projectPoints(objpoints[i].detach().numpy(), rvecs[i].detach().numpy(), tvecs[i].detach().numpy(), mtx.detach().numpy(), dist.detach().numpy())
        imgpoints2 = torch.tensor(imgpoints2, dtype=torch.float32).reshape(-1, 2)
        error = torch.sum((imgpoints[i] - imgpoints2) ** 2)
        total_error += error
    total_error.requires_grad_(True)
    return total_error

In [31]:
params = torch.cat((rvecs.flatten(), tvecs.flatten()), dim=0)
params.requires_grad_(True)

epochs = 100
learning_rate = 1e-2

optimizer = Adam([params], lr=learning_rate, weight_decay=1e-3)

In [32]:
for e in range(epochs):
    optimizer.zero_grad()
    error = reprojection_error(
        params[:len(rvecs)*3].reshape(len(rvecs), 3), 
        params[len(rvecs)*3:].reshape(len(tvecs), 3),
        objpoints, imgpoints, mtx, dist
        )
    error.backward()
    optimizer.step()
    print(f"Epoch {e}: Reprojection Error = {error.item()}")

Epoch 0: Reprojection Error = 206186217472.0
Epoch 1: Reprojection Error = 206186217472.0
Epoch 2: Reprojection Error = 206186217472.0
Epoch 3: Reprojection Error = 206186217472.0
Epoch 4: Reprojection Error = 206186217472.0
Epoch 5: Reprojection Error = 206186217472.0
Epoch 6: Reprojection Error = 206186217472.0
Epoch 7: Reprojection Error = 206186217472.0


Epoch 8: Reprojection Error = 206186217472.0
Epoch 9: Reprojection Error = 206186217472.0
Epoch 10: Reprojection Error = 206186217472.0
Epoch 11: Reprojection Error = 206186217472.0
Epoch 12: Reprojection Error = 206186217472.0
Epoch 13: Reprojection Error = 206186217472.0
Epoch 14: Reprojection Error = 206186217472.0
Epoch 15: Reprojection Error = 206186217472.0
Epoch 16: Reprojection Error = 206186217472.0
Epoch 17: Reprojection Error = 206186217472.0
Epoch 18: Reprojection Error = 206186217472.0
Epoch 19: Reprojection Error = 206186217472.0
Epoch 20: Reprojection Error = 206186217472.0
Epoch 21: Reprojection Error = 206186217472.0
Epoch 22: Reprojection Error = 206186217472.0
Epoch 23: Reprojection Error = 206186217472.0
Epoch 24: Reprojection Error = 206186217472.0
Epoch 25: Reprojection Error = 206186217472.0
Epoch 26: Reprojection Error = 206186217472.0
Epoch 27: Reprojection Error = 206186217472.0
Epoch 28: Reprojection Error = 206186217472.0
Epoch 29: Reprojection Error = 20618